In [ ]:
!pip install telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 22.2 MB/s eta 0:00:00


In [ ]:
!pip install -q transformers datasets accelerate peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 19.5 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login("token")


In [ ]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,                         # 8-bit quant
    llm_int8_enable_fp32_cpu_offload=True      # ✅ allow CPU offload safely
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",                         # spread across GPU+CPU
    low_cpu_mem_usage=True
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
import telebot
from telebot import types

# def generate_text(prompt: str, max_len: int = 128) -> str:
#     inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
#     inputs = {k: v.to(model.device) for k, v in inputs.items()}
#     outputs = model.generate(**inputs, max_length=max_len)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ================== BOT SETUP ==================
bot_token = "8490280338:AAGRcnOmW3QxJB14vkGxMDKrkv2ikna8aug"
bot = telebot.TeleBot(bot_token)

# Store user states
user_states = {}

# ================== AI HELPERS ==================
def fix_grammar(sentence: str) -> str:
    prompt = f"You're an expert English tutor. Fix grammar mistakes in this sentence:\n\n{sentence}\n\nCorrected version:"
    return generate_text(prompt)

def generate_text(prompt):
    chat_prompt = f"[INST] {prompt.strip()} [/INST]"
    inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Zero-shot: Find resources for a topic
def test_english(level: str):
    prompt = f"""
    You are an English teacher. Create MCQs (Multiple Choice Questions) for grammar and vocabulary.
    Each question should have 4 options and the correct answer should be clearly marked.

    Example 1 (Grammar):
    Q: Choose the correct sentence.
    a) He go to school every day.
    b) He goes to school every day. ✅
    c) He going to school every day.
    d) He gone to school every day.

    Example 2 (Vocabulary):
    Q: What is the synonym of 'happy'?
    a) Sad
    b) Angry
    c) Joyful ✅
    d) Lonely

    Now generate 3 new questions (mix grammar and vocabulary).
    for english level: {level}
    """
    return generate_text(prompt)

def suggest_resources(text: str) -> str:
    prompt = f"""You are an English teacher. Provide study resources for improving
    grammar and vocabulary. Include practice tips and example sentences about topic: {text}."""
    return generate_text(prompt)

# ================== TELEGRAM FLOWS ==================
@bot.message_handler(commands=['start'])
def start_menu(message):
    chat_id = message.chat.id
    markup = types.InlineKeyboardMarkup()
    btn1 = types.InlineKeyboardButton("🛠 Fix Grammar", callback_data="fix")
    btn2 = types.InlineKeyboardButton("✅ Test Your English", callback_data="test")
    btn3 = types.InlineKeyboardButton("📚 Resources", callback_data="resources")
    markup.add(btn1, btn2, btn3)
    bot.send_message(chat_id, "Please select an option:", reply_markup=markup)

# --- Levels Menu ---
def english_levels(message):
    chat_id = message.chat.id
    markup = types.InlineKeyboardMarkup()
    for level in ["A1", "A2", "B1", "B2", "C1", "C2"]:
        markup.add(types.InlineKeyboardButton(level, callback_data=level))
    bot.send_message(chat_id, "Please select a level:", reply_markup=markup)

# --- Handle Option Selection ---
@bot.callback_query_handler(func=lambda call: True)
def handle_option(call):
    chat_id = call.message.chat.id

    if call.data == "fix":
        user_states[chat_id] = "fix"
        bot.send_message(chat_id, "✍️ Send me the sentence you want me to fix.")

    elif call.data == "test":
        user_states[chat_id] = "test"
        english_levels(call.message)

    elif call.data == "resources":
        user_states[chat_id] = "resources"
        bot.send_message(chat_id, "✍️ Send me a text, and I'll suggest resources.")

    elif call.data in ["A1", "A2", "B1", "B2", "C1", "C2"]:
        level = call.data
        user_states[chat_id] = f"test_{level}"
        bot.send_message(chat_id, f"📖 Level {level} selected. Generating your English test...")
        test = test_english(level=level)
        bot.send_message(chat_id, f"📝 Your English Test:\n\n{test}")

# --- Handle Text Input After Choosing ---
@bot.message_handler(func=lambda message: True)
def handle_text(message):
    chat_id = message.chat.id
    text = message.text

    if chat_id not in user_states:
        bot.send_message(chat_id, "⚠️ Please type /start and choose an option first.")
        return

    state = user_states[chat_id]

    if state == "fix":
        fixed = fix_grammar(text)
        bot.send_message(chat_id, f"🔧 Fixed Sentence:\n{fixed}")

    elif state.startswith("test_"):
        bot.send_message(chat_id, "✅ Please answer the test by sending option letters (A, B, C...).")

    elif state == "resources":
        recs = suggest_resources(text)
        bot.send_message(chat_id, f"📚 Recommended Resources:\n{recs}")

    user_states.pop(chat_id, None)

# ================== RUN BOT ==================
bot.infinity_polling()


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
